In [45]:
!pip install keras
!pip install tensorflow
!pip install sklearn

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py bdist_wheel for sklearn: started
  Running setup.py bdist_wheel for sklearn: finished with status 'done'
  Stored in directory: C:\Users\dlt\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Get the vector representations of each word according to Glove

In [1]:
import numpy as np

embedding_path = "glove.twitter.27B.100d.txt" ## change
# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    try:
        file = open(embedding_path, "r", encoding='utf8')
        if (file):
            word2vec = dict()
            split = file.read().splitlines()
            for line in split:
                key = line.split(' ',1)[0] # the first word is the key
                value = np.array([float(val) for val in line.split(' ')[1:]])
                word2vec[key] = value
            file.close()
            return (word2vec)
    except Exception as e:
        print(e)
w2v = get_word2vec(embedding_path)


In [2]:
print(type(w2v))

<class 'dict'>


In [3]:
print(w2v['test'])

[ 4.5849e-01  1.4165e-01  8.3564e-03  4.0577e-02  4.4195e-01 -1.1454e-01
  2.0338e-01 -1.3377e-01 -6.1458e-02  1.1220e+00  2.1163e-01 -5.4524e-01
 -3.0683e+00 -1.4878e-01 -1.4466e-01 -5.9170e-01  9.8828e-03  6.1911e-01
 -8.6552e-01 -7.0807e-01 -3.6516e-01  2.1672e-02 -3.5861e-01  4.8487e-01
 -5.6540e-01 -5.3430e-01 -3.4473e-01 -1.9008e-01  9.3164e-02  4.8268e-01
 -1.8350e-01 -1.1083e-01 -3.3814e-01  6.3688e-01  6.2268e-01  8.2295e-01
  4.3433e-01  6.5400e-02  7.2992e-01  2.6747e-02 -8.5995e-01 -3.0779e-02
 -5.6146e-01  1.8875e-01  7.8385e-01 -2.1126e-01 -2.1223e-01  9.0534e-02
  3.2052e-02 -7.1102e-01  9.1668e-01  5.4492e-01 -2.8641e-01  1.3797e-02
  8.2005e-01 -2.7975e-02  3.5742e-01 -7.2516e-01 -4.8604e-02 -2.5878e-01
 -8.5122e-01  1.9878e-01  1.2547e+00  3.9707e-01 -1.4316e-01  1.1500e-02
 -7.1699e-02 -7.9911e-01  8.2668e-01 -7.4625e-01 -3.7770e-01  1.0727e+00
 -3.8060e-01  1.9366e-01  2.1991e-01 -5.4513e-01  6.6331e-01 -4.2717e-01
  2.5983e-01 -6.0330e-01  9.6712e-01  5.5068e-01  4

### Tokenization

In [4]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')
tknzr = TweetTokenizer()

stopwords = {'then', 'to', 'himself', 'you', 'as', 'it', 'am', 'whom', 'was', "you're", 'll', 'once', 'from', 'does', 's', 'nor', 'their',
             'ma', 'a', 'while', 'any', 'yours', 'itself', 'during', 'by', 'where', 'its', 'on', 'no', 'yourselves', 'she', 'here', 'myself',
             'do', 'about', 'can', 'we', 'm', 'and', 'there', 'should', 'who', 'those', 'these', 'through', 't', 'below', 'what', 'your',
             'how', 'just', 'down', 'own', 'y', 'hers', 'is', 'so', 'being', 'the', "you've", "it's", 'more', 're', 'them', 'have', 'off',
             'been', 'him', 'this', 'will', 'each', 'all', 'herself', 'up', 'has', 'ours', 'because', "you'll", "you'd", 'his', 'very', 'are',
             'having', 'which', 'that', 'into', 'if', 'themselves', 'of', 'theirs', 'for', 'out', 'd', 'yourself', 'o', 'they', 'our', 'over',
             'he', 've', 'doing', 'some', "that'll", 'now', 'were', 'than', 'ourselves', 'above', 'did', 'under', 'me', 'her', 'until','my',
             'further', 'had', 'other', "she's", 'an', 'between', 'but', 'such', 'with', 'be', 'i', 'at', 'when', 'why', 'or', 'before', 'in'}


def get_tokens(sentence):
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


def get_lemma(word): # lemmatize and lower the word
    lemma = wn.morphy(word)
    if lemma is None:
        return word.lower()
    else:
        return lemma.lower()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dlt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Get training text as input and label as output

In [98]:
import json
source_file_name = 'test_c.json'
source_file = open(source_file_name,'r', encoding='utf8')
source_sentences = []
true_sentences = []
false_sentences = []
labels = []
for line in source_file:
    source_json = json.loads(line)    
    if source_json['text']:
        if source_json['label'] == "0":
            true_sentences.append(source_json['text'])
        else:
            false_sentences.append(source_json['text'])
        source_sentences.append(source_json['text'])
        labels.append(source_json['label'])

In [99]:
print(len(source_sentences))
print(len(true_sentences))
print(len(false_sentences))

620000
310000
310000


### Get Credbank test set input

In [62]:
def get_test_set(file_name):
    test_list = []
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in f:
            test_list.append(line.replace('\n','').lower())
    return test_list

In [127]:
test_list_cred = get_test_set('cred_realnews_processed.txt')

### Get Pheme test set input

In [82]:
def get_pheme_set(file_name):
    text_list = []
    label_list = []
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in f:
            line = json.loads(line)
            text_list.append(line['text'].lower())
            label_list.append(line['label'])
    return text_list, label_list

In [83]:
test_list_pheme, label_list_pheme = get_pheme_set('pheme_processed.json')

In [84]:
print(test_list_pheme[0])
print(label_list_pheme[0])

there are no verses in the quran about me wanting anyone to hold hostages in a chocolate shop in sydney, you terrorist fucks.
0


### Get Combined train and test set input

In [93]:
# def get_combined_set(file_name1, file_name2, num1, num2):
#     test_list = []
#     with open(file_name1, 'r', encoding='utf-8') as f1:
#         i=0
#         if i < num1:
#         for line in f1:
#             source_json = json.loads(line)    
#             if source_json['text']:
#                 source_sentences.append(source_json['text'])
#                 labels.append(source_json['label'])
#     with open(file_name2, 'r', encoding='utf-8') as f2:
#         i=0
#         for line in f2:
#             if i < num:
#                 test_list.append(line.replace('\n','').lower())
#             if i == num2 - 1:
#                 break
#             i += 1
#     return test_list

In [186]:
def get_combined_set(real_texts1, real_texts2, fake_texts, real_num , fake_num):
    train_set = []
    test_set = []
    train_y = []
    test_y = []
    test_set += real_texts1[:len(real_texts1)-real_num]
    print("faknewsnet real test: ", len(test_set))
    test_set += real_texts2[:real_num]
    print("real test: ", len(test_set))
    test_set += fake_texts[:fake_num]
    print("test: ", len(test_set))
    test_y = [0] * len(real_texts1) + [1]* fake_num
    train_set += real_texts1[len(real_texts1)-real_num:]
    print("fakenewsnet train real: ", len(train_set))
    train_set += real_texts2[real_num:]
    print("real train: ",len(train_set))
    train_set += fake_texts[fake_num:]
    train_y = [0] * len(real_texts2) + [1]* (len(fake_texts)-fake_num)

    return train_set, test_set, train_y, test_y

In [187]:
print(len(true_sentences))
print(len(test_list_cred))
print(len(false_sentences))

310000
499277
310000


In [188]:
train_combined_ordered,test_combined_ordered,train_y_co,test_y_co = get_combined_set(true_sentences, test_list_cred, false_sentences , 150000, 150000)

faknewsnet real test:  160000
real test:  310000
test:  460000
fakenewsnet train real:  150000
real train:  499277


In [165]:
print(len(train_combined_ordered))
print(len(test_combined_ordered))
print(len(train_y_co))
print(len(test_y_co))

659277
460000
659277
460000


### Shuffle the inputs and output labels randomly

In [8]:
training_dict = {}
for i in range(len(source_sentences)):
    training_dict[source_sentences[i]] = labels[i] 

In [9]:
import random
training_list = list(training_dict.items())
random.shuffle(training_list)

processed_texts, processed_labels = [], []
for text, label in training_list:
    processed_texts.append(text.rstrip())
    processed_labels.append(label)

### Tokenize Tweets text and make it a token library

In [11]:
token_list = []
for text in processed_texts:
    token_list += get_tokens(text)

In [12]:
print(len(token_list))
print(token_list[:100])

6376498
['scary', 'scary', 'stuff', 'wake', 'america', 'wake', 'people', 'common', 'sense', 'hit', 'right', 'face', 'don', 'ignore', 'wi', 'news', 'income', 'poverty', 'health', 'insurance', 'coverage', 'united', 'states', 'this', 'report', 'present', 'data', 'sorry', 'vicki', 'say', 'us', 'something', 'else', 'please', 'don', 'leave', 'make', 'show', 'breakingnews', 'princess', 'charlotte', 'net', 'worth', 'is', 'more', 'than', 'billion', 'more', 'than', 'prince', 'george', 'kanye', 'west', 'mysteriously', 'delete', 'twitter', 'instagram', 'account', 'samheughan', 'alphamalemadness', 'wait', 'til', 'get', 'home', 'holiday', 'binge', 'watch', 'shadowhunters', 'netflix', 'aus', 'superpumped', 'bookishthings', 'chris', 'pratt', 'dating', 'mystery', 'blonde', 'gossip', 'cop', 'hell', 'bell', 'sir', 'one', 'hundred', 'percent', 'correct', 'damnit', 'janet', 'breaking', 'tom', 'petty', 'reportedly', 'rush', 'hospital', 'full', 'cardiac', 'arrest', 'after', 'suffering', 'heart']


### Get the average, max, mid length for a more precise padding

In [13]:
sent_len_list = []
for sent in processed_texts:
    sent_len_list.append(sent.count(' '))
sent_len_list = sorted(sent_len_list)
print(len(sent_len_list))
print(sum(sent_len_list)/len(sent_len_list))
print(max(sent_len_list))
print(sent_len_list[int(len(sent_len_list)/2)])

617418
16.334356951044512
75
17


In [14]:
print(sent_len_list[15000])

2


### Encode sentences using the tokenized document

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# prepare tokenizer
t = Tokenizer(oov_token="<UKN>")
t.fit_on_texts(token_list)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(processed_texts)
# pad documents to a max length of 4 words
max_length = 50
X = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [68]:
encoded_testset = t.texts_to_sequences(test_list)
X_test = pad_sequences(encoded_testset, maxlen=max_length, padding='post')

In [85]:
encoded_testset_pheme = t.texts_to_sequences(test_list_pheme)
X_test_pheme = pad_sequences(encoded_testset_pheme, maxlen=max_length, padding='post')

In [96]:
encoded_true = t.texts_to_sequences(true_sentences)
X_true = pad_sequences(encoded_true, maxlen=max_length, padding='post')

In [100]:
encoded_false = t.texts_to_sequences(false_sentences)
X_false = pad_sequences(encoded_false, maxlen=max_length, padding='post')

In [147]:
encoded_train_combined_ordered = t.texts_to_sequences(train_combined_ordered)
X_train_co = pad_sequences(encoded_train_combined_ordered, maxlen=max_length, padding='post')

In [170]:
encoded_test_combined_ordered = t.texts_to_sequences(test_combined_ordered)
X_test_co = pad_sequences(encoded_test_combined_ordered, maxlen=max_length, padding='post')

In [121]:
from sklearn.model_selection import train_test_split
Y_true = [0]*len(X_true)
Y_test_cred = [0]*len(X_test) 
Y_false = [1]*len(X_false)
X_true_train1,X_true_test1, Y_true_train1, Y_true_test1 =  train_test_split(X_true, Y_true,test_size =0.5,random_state= 4 )
X_true_train2,X_true_test2, Y_true_train2, Y_true_test2 =  train_test_split(X_test, Y_test_cred,test_size =0.5,random_state= 4 )
X_false_train,X_false_test, Y_false_train, Y_false_test =  train_test_split(X_false, Y_false,test_size =0.2,random_state= 4 )
X_combined_train= np.concatenate((X_true_train1, X_true_train2[:int((len(X_true)/2))]), axis=0)
X_combined_train= np.concatenate((X_combined_train, X_false_train), axis=0)
print(len(X_combined_train))
X_combined_test= np.concatenate((X_true_test1, X_true_train2[int((len(X_true)/2)):]), axis=0)
X_combined_test= np.concatenate((X_combined_test, X_true_test2 ), axis=0)
X_combined_test= np.concatenate((X_combined_test, X_false_test ), axis=0)
print(len(X_combined_test))
Y_combined_train= np.concatenate((Y_true_train1, Y_true_train2[:int((len(X_true)/2))]), axis=0)
Y_combined_train= np.concatenate((Y_combined_train, Y_false_train), axis=0)
print(len(Y_combined_train))
Y_combined_test= np.concatenate((Y_true_test1, Y_true_train2[int((len(X_true)/2)):] ), axis=0)
Y_combined_test= np.concatenate((Y_combined_test, Y_true_test2  ), axis=0)
Y_combined_test= np.concatenate((Y_combined_test, Y_false_test ), axis=0)
print(len(Y_combined_test))

558000
561277
558000
561277


In [ ]:
print(len(X_test_pheme))
print(X_test_pheme[10])

In [86]:
print(len(X))
print(X[10])
print(len(X_test))
print(X_test[10])
print(len(X_test_pheme))
print(X_test_pheme[10])

617418
[  238     1  2154   852  2460  3991   125  1273   182 10942  7610  6339
    17  2154   852     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
499277
[  137   467  1425  1234 13188   571   928    56   194     2  4009  5307
     5  1602     1  7531    40     1     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
7878
[ 162    2    1 6395    3  345  264 7226   26  624  698   40    2  770
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


### Transform label to unique number

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y_new = processed_labels
Y_new = le.fit_transform(Y_new)

In [18]:
print(Y_new[:100])

[1 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 0
 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 0
 1 1 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0]


In [90]:
Y_pheme = label_list_pheme
Y_pheme = le.fit_transform(Y_pheme)

In [91]:
print(Y_pheme[:100])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [166]:
Y_train_co = train_y_co
Y_train_co = le.fit_transform(Y_train_co)

In [167]:
print(len(Y_train_co))

659277


### Get the Word2Vec representations of tokens

In [19]:
# get the embedding matrix from the embedding layer
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = w2v.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [20]:
print(len(embedding_matrix[5]))

100


In [21]:
# main model
from keras import Input, Model
from keras.layers import Bidirectional, Embedding, TimeDistributed, Dense, LSTM, Flatten

inpt = Input(shape=(max_length,))
print(inpt)
model = Embedding(vocab_size,100, weights=[embedding_matrix],input_length=max_length)(inpt)
# Embedding Layer: input_dimension=vocabulary size, output_dimension(to dense layer,should be consistent with embedding matrix?),
# weights using pretrained embeddings, max length of tokenized sentence input)
print(model)
model = Bidirectional(LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
# LSTM: units=100->output dimension to next layer, dropout->prevent overfitting, 
print(model)
model = TimeDistributed(Dense(100,activation='relu'))(model) 
# activation function -> transform weighted sum to activation, Rectified Linear Unit overcomes the vanishing gradient problem
print(model)
model = Flatten()(model)
print(model)
model = Dense(100,activation='relu')(model)
print(model)
outpt = Dense(3,activation='softmax')(model)
print(outpt)
model = Model(inpt,outpt)
print(model)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

Tensor("input_1:0", shape=(?, 50), dtype=float32)
Instructions for updating:
Colocations handled automatically by placer.
Tensor("embedding_1/embedding_lookup/Identity:0", shape=(?, 50, 100), dtype=float32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("bidirectional_1/concat:0", shape=(?, ?, 200), dtype=float32)
Tensor("time_distributed_1/Reshape_1:0", shape=(?, 50, 100), dtype=float32)
Tensor("flatten_1/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("dense_2/Relu:0", shape=(?, 100), dtype=float32)
Tensor("dense_3/Softmax:0", shape=(?, 3), dtype=float32)


In [30]:
X_train,X_ev, Y_train, Y_ev =  train_test_split(X, Y_new,test_size =0.1,random_state= 4 )

In [31]:
print(X_train[:100])
print(Y_train[:100])
print(X_ev[:100])
print(Y_ev[:100])

[[  233  1615   412 ...     0     0     0]
 [21323   927  2259 ...     0     0     0]
 [  328    82     2 ...     0     0     0]
 ...
 [21323 32431   107 ...     0     0     0]
 [    2   164  1217 ...     0     0     0]
 [ 3208    31   325 ...     0     0     0]]
[1 1 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 1 0 0 1 1 1 0
 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1
 1 0 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1]
[[9889 1602   43 ...    0    0    0]
 [2437 5096   76 ...    0    0    0]
 [ 893  856  345 ...    0    0    0]
 ...
 [ 858  858  858 ...    0    0    0]
 [  34  321   40 ...    0    0    0]
 [ 693  318 6872 ...    0    0    0]]
[0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 0 0
 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 0 0 0 1 0 1 0 0 0
 1 1 0 0 0 1 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1]


In [169]:
model.fit(X_train_co,Y_train_co,validation_split=0.1, nb_epoch = 1, verbose = 2)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 593349 samples, validate on 65928 samples
Epoch 1/1
 - 3592s - loss: 0.0652 - acc: 0.9728 - val_loss: 0.6029 - val_acc: 0.7434


In [32]:
# evaluate the model
loss, accuracy = model.evaluate(X_ev, Y_ev, verbose=2)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 95.729001


In [53]:
from sklearn.metrics import classification_report, confusion_matrix
Y_ev_pred = model.predict(X_ev)
Y_ev_pred = np.array([np.argmax(pred) for pred in Y_ev_pred])

In [55]:
print(len(Y_ev_pred))
print(len(Y_ev))
print(Y_ev[0])
print(type(Y_ev))
conf_matrix = confusion_matrix(Y_ev, Y_ev_pred)
print(conf_matrix)
precision0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[1][0])
precision1 = conf_matrix[1][1]/(conf_matrix[0][1] +conf_matrix[1][1])
recall0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[0][1])
recall1 = conf_matrix[1][1]/(conf_matrix[1][0] +conf_matrix[1][1])
f1_score0 = 2 * (precision0 * recall0) / (precision0 + recall0)
f1_score1 = 2 * (precision1 * recall1) / (precision1 + recall1)
print(precision0, precision1, recall0, recall1, f1_score0, f1_score1)

61742
61742
0
<class 'list'>
[[29296  1703]
 [  934 29809]]
0.969103539530268 0.945957095709571 0.9450627439594825 0.9696191002829913 0.9569321726632805 0.9576419564693599


### Predict credbank test set label

In [70]:
Y_test = model.predict(X_test)

In [71]:
Y_test = np.array([np.argmax(pred) for pred in Y_test])

In [72]:
print(Y_test)

[1 0 1 ... 0 1 1]


In [75]:
true_count = 0
for label in Y_test:
    if label == 0:
        true_count += 1
print(true_count/len(test_list))

0.36064749627962034


In [87]:
Y_test_pheme = model.predict(X_test_pheme)

In [88]:
Y_test_pheme = np.array([np.argmax(pred) for pred in Y_test_pheme])

In [92]:
print(len(Y_test_pheme))
print(len(Y_pheme))
print(Y_test_pheme[0])
conf_matrix = confusion_matrix(Y_pheme, Y_test_pheme)
print(conf_matrix)
precision0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[1][0])
precision1 = conf_matrix[1][1]/(conf_matrix[0][1] +conf_matrix[1][1])
recall0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[0][1])
recall1 = conf_matrix[1][1]/(conf_matrix[1][0] +conf_matrix[1][1])
f1_score0 = 2 * (precision0 * recall0) / (precision0 + recall0)
f1_score1 = 2 * (precision1 * recall1) / (precision1 + recall1)
print(precision0, precision1, recall0, recall1, f1_score0, f1_score1)

7878
7878
1
[[ 845 3094]
 [1122 2817]]
0.4295882053889171 0.476569108441888 0.2145214521452145 0.7151561309977151 0.2861496782932611 0.5719796954314721


### Predict combined test set label

In [123]:
Y_combine_pred = model.predict(X_combined_test)
Y_combine_pred = np.array([np.argmax(pred) for pred in Y_combine_pred])

In [124]:
print(len(Y_combine_pred))
print(len(Y_combined_test))
print(Y_combine_pred[0])
conf_matrix = confusion_matrix(Y_combined_test, Y_combine_pred)
print(conf_matrix)
precision0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[1][0])
precision1 = conf_matrix[1][1]/(conf_matrix[0][1] +conf_matrix[1][1])
recall0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[0][1])
recall1 = conf_matrix[1][1]/(conf_matrix[1][0] +conf_matrix[1][1])
f1_score0 = 2 * (precision0 * recall0) / (precision0 + recall0)
f1_score1 = 2 * (precision1 * recall1) / (precision1 + recall1)
print(precision0, precision1, recall0, recall1, f1_score0, f1_score1)

561277
561277
0
[[485624  13653]
 [  3524  58476]]
0.9927956364944761 0.8107141371709021 0.9726544583467694 0.9431612903225807 0.9826218478893188 0.8719367176374984


In [189]:
Y_pred_co = model.predict(X_test_co[:100])

In [190]:
print(Y_pred_co[:3])
Y_pred_co = np.array([np.argmax(pred) for pred in Y_pred_co])

[[0.9714225  0.02857753 0.        ]
 [0.23486398 0.76513594 0.        ]
 [0.09289969 0.9071003  0.        ]]


In [172]:
print(len(Y_pred_co))
print(len(test_y_co))
print(Y_pred_co[0])
conf_matrix = confusion_matrix(test_y_co, Y_pred_co)
print(conf_matrix)
precision0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[1][0])
precision1 = conf_matrix[1][1]/(conf_matrix[0][1] +conf_matrix[1][1])
recall0 = conf_matrix[0][0]/(conf_matrix[0][0] +conf_matrix[0][1])
recall1 = conf_matrix[1][1]/(conf_matrix[1][0] +conf_matrix[1][1])
f1_score0 = 2 * (precision0 * recall0) / (precision0 + recall0)
f1_score1 = 2 * (precision1 * recall1) / (precision1 + recall1)
print(precision0, precision1, recall0, recall1, f1_score0, f1_score1)

460000
460000
0
[[306856   3144]
 [ 14690 135310]]
0.9543144682253861 0.9772920970141707 0.989858064516129 0.9020666666666667 0.9717613602176247 0.9381738509433046
